# Quantitative Finance
**Team**: Davi Barreira, Gabriel Doc, Pedro Mendes

Consider the following problem. We wish to liquidate a significant amount of an asset, such that
our actions affect it's price.
If we just try to sell everything at the current market price, our order will walk the book, and thus,
we'll end up selling at a price possibly much lower. To avoid this problem, we wish to break our order
in smaller orders, and "slowly" sell the asset, so that the Limit Order Book can be "replenished".
The problem now is, if you take to long to sell the asset, you are subjected to the market's variation,
and the asset's price might drop (which is probably what you expected, since you wanted to sell!).

Hence, we are looking for an optimal strategy to sell the asset. How fast should we sell it?
Also, we consider that our time-horizon is finite, i.e. we wish to liquidate all the assets
at most at time T (this could be an hour, a day, a month, etc).

### Modeling the Problem
Let's define some of the variables:
* $\nu$ - The strategy to liquidate the asset;
* $\nu_t$ - The rate of which we sell the asset at time $t$.
* $\mu_t$ - The net order flow, i.e. the rate at which the market is buying/selling the asset;
* $X_t^\nu$ - The cash flow of the agent at time $t$ using strategy $\nu$;
* $S_t^\nu$ - The price of the asset at time $t$ under strategy $\nu$;
* $Q_t^\nu$ - Amount of "shares" of the asset the agent is holding at time $t$ with strategy $\nu$;

Our modelling assumptions are the following.
First, we'll assume that the both our action and the market's affect the price of the assets linearly and equally,
and that the price of the asset has a "natural" variability modeled as a Brownian motion with variance $\sigma$. Thus,
the price of the asset is given by

$$
dS^\nu_t = \sigma dW_t + b(\mu_T - \nu_t)dt,
$$

where $b \in \mathbb R_+$, and $W_t$ is the Brownian motion.

The net order flow $\mu_t$ will be modelled as two independent Poisson processes, one for buy orders and another for
sell orders with

$$
d\mu_t = -\kappa \mu_t dt + \eta (dL_t^+ - dL_t^-)
$$

where $\eta$ is the impact of the increase/decrease of order flow, and $\kappa$ is a rate of decay of the jumps.
Our last simplifying assumption is that, when we sell at a rate $\nu_t$, we always walk the book a bit, which
is approximated by a linear equation $k \nu_t$ to model such impact, with $k>0$.

Thus, the cash process of the
agent is given by:

$$
d X_t^\nu = (S^\nu_t - k \nu_t) \nu_t dt.
$$

Note that if we assumed that we always sell the the midprice, then cash process would be simply $dX_t^\nu = S_t^\nu \nu_t dt$, i.e. the amount
of money we get is the amount we trade times the price. The amount of shares we have can be computed as:

$$
dQ_t^\nu = - \nu_t dt.
$$

Lastly, we need to know what function we are going to try to optimize. This is a bit controversial. We might
think that what we want to do is find $\nu$ that maximizes $E[X^\nu_t]$. But, if this was the case, the strategy
is not penalized if it's too slow. Hence, instead of $E[X^\nu_t]$, we'll try to maximize

$$
E[X^\nu_t + Q_T^\nu(S^\nu_T - \alpha Q^\nu_T) - \phi \int_t^T (Q^\nu_s)^2 ds].
$$

Note that we added two extra terms. The first one was $Q_T^\nu(S^\nu_T - \alpha Q^\nu_T)$. This term penalizes
the strategy if there is still some of the asset left by the end of time $T$. When this happens, the agent
has to sell everthing at market price. But if we do that, the strategy is penalized by the amount of shares times
a parameter $\alpha > 0$.
Secondly,  $\phi \int_t^T (Q^\nu_s)^2 ds$, which penalizes the strategy by how long it holds the asset. This is controlled
by a parameter $\phi>0$. The larget $\phi$, the quicker the agent wants to get rid of the asset.